In [25]:
# I bet there is an easier way to do this since the pieces always fall in the same order 
# and the wind repeats, eventually this has to be cyclical. Finding that cycle though ...
# maybe for a later attempt when cleaning up.

# or now, since that's the only way to solve it. 

# wrote this code to first attempt a sliding window and only represent part of the blocks/chasm.
# was still going to take forever, so added code to print out info at various points in time
# as well as to find the periodic cycle I was thinknig about above.

# this code is more of a 'show me data so I can do the calculation' than an actual
# 'punch everything in and it spits out an answer' type thing.

# handy imports to have
%matplotlib inline
from sympy import symbols, Eq, solve
import math
from itertools import combinations_with_replacement , permutations, combinations, count, chain
import numpy as np
import time
import functools
from tqdm.notebook import trange, tqdm
import ipywidgets as widgets
import matplotlib.pyplot as plt
from collections import Counter, deque
import more_itertools as mit
import string
import ast
import networkx as nx

In [46]:
shape_names_dict = {0: 'square', 1:'Horiz Bar', 2:'Plus', 3:'Backwards L', 4:'Vert Bar'}

class block:
    
    def __init__(self, sequence_num, highest):
        self.shape = sequence_num % 5
        self.shape_name = shape_names_dict[self.shape]
        
        match self.shape:
            
            case 0:   # square
                self.coords = [(highest+4, i) for i in range(3, 5)] + [(highest+5,i) for i in range(3,5)]
            case 1:  # horiz bar
                self.coords = [(highest+4, i) for i in range(3, 7)]
            case 2:  # Plus
                self.coords = [(highest+5, i) for i in range(3, 6)] + [(highest+4, 4), (highest+6, 4)]
            case 3:  # backwards L
                self.coords = [(highest+4, i) for i in range(3, 6)] + [(i, 5) for i in range(highest+5, highest+7)]
            case 4: 
                self.coords = [(i, 3) for i in range(highest+4, highest+8)]
            case _:
                assert False, 'got an invlalid shape number/name'
    
    def get_coords(self):
        return self.coords
    
    def can_move_left(self, c, offset):
        for y,x in self.coords:
            if c[y-offset][x-1] == '|' or c[y-offset][x-1] == '#':
                return False
        return True
    
    def can_move_right(self, c, offset):
        for y,x in self.coords:
            if c[y-offset][x+1] == '|' or c[y-offset][x+1] == '#':
                return False
        return True
    
    def is_placed(self, c, offset):
        for y,x in self.coords:
            #print(y, offset, y-offset-1)
            if c[y-offset-1][x] == '|' or c[y-offset-1][x] == '#' or c[y-offset-1][x] == '-':
                return True
        return False
        
    
    def move(self, c, direction, offset):
        if direction == '<':   # if blowing left
            if self.can_move_left(c, offset):
                self.coords = [(y,x-1) for y,x in self.coords]
                return True
        elif direction == '>':
            if self.can_move_right(c, offset):
                self.coords = [(y,x+1) for y,x in self.coords]
                return True
        elif direction == 'down':
            if not self.is_placed(c, offset):
                self.coords = [(y-1,x) for y,x in self.coords]
                return True
        
        return False
            
        
        

In [64]:
# sample input
wind_input = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

In [83]:
# puzzle input
wind_input = '>>><<<>><<<>>><<<>>><<<>><<><>>><<>>><>><<<>>><<><>>><<<<>>>><>><<>>>><>>>><<<<>>>><<<>>><>><<>><<<>><<<<><<>><<><<<>><<>>><<<>>>><>>><>><><<<<><>><><<>>>><<><<<<><<<><<>>>><<><<>><<<<>>>><>>>><>><><<<>>>><>>>><>><<>>>><>>>><<><<<<>><<>>>><>>><>><>><<>><<<<>>>><<<>><<<><><<<<>><<><<<<><<<><<<<>>>><<><<<>>><<>><>><<><<<<>>>><<<>>>><<<<>><<<<>><<>>>><<><>><>>>><<<<>>>><<>>><<<><<<<>>><>><<<>>><<<>>>><<<>>>><>>>><>>><<<<>>>><<<>>><<<<>>><><<>>><<<>>>><<<>>>><>>>><<<<>><<>>><<<<><>>>><><<<<>>><<<<>>>><<<>>>><<>>><>><<>><<>><<>>><<>>>><>>><<<><<<>><<<<>>><<<>>><<<><>>><<<<><>><<<>>>><><<<<><<>><<<<>><<<>>><>><<><><<<>><<<>>><<>>>><<>><><<>>>><<<<>>><<<>><<<<>><<><<<>>>><>>><>><<<>>>><<<>>><>><<<<><><<>><<<>><<<<>>>><<<<>>><<<<>><>>>><>>><>>><<>>><<<<>><<><<<<>>>><<<<>>>><<<><<>><><<<>>><>><<>><<<>>><>>><<<<>><>><><<><>><>>><>>>><<<>>><<<<>>>><<<>>>><<<>><<<<>>>><<<>>><><<>>><>>><<<>>><<<<><>>>><<<><>>>><>>><<>>><>><<<<>>><<>><<<<>><<>>>><<<><<><><<<>><<<<>>><<<><<<<>>>><<<<>>>><<<<>><<>>><<<><<<<><<<>>><><<<<><>><<><<<<>>><<>><>><<<><<<>>><<>><<<<><<<>><<<<>>><<>>><<<>>>><<<>><<<>>>><><<>><>>><<<>><><>><<>>><<<<>>><<>>>><>><<><<<>>>><<<>>><<>><<<<>>><<<<><<<<><<>>>><>><><<<>>>><<<<>><<<<>><>><<>><<<>><><<<<>>>><<<<>>>><<<>>>><>><<>><<<><<<<>><<<<><<<><<<>>><<<>><>>><<>><<><<>><<>><<><<<><<<<>>>><<<><>>>><<>><<>>><>>><<>>><><<>>><<<>>>><<<<>>>><><<<>>><<>>>><<<<><>>><>><<>><<<><><>>><<<>>>><>>><<><>>><<>>><<<>>><<<><<<>>><<<<><<<<>>>><<<>>>><<<<>>>><<<>>><<>><<>>><<<<>><><<<><<><<<>>>><<<<>>><>><<<<><<<>>>><<<<>>>><<<>><>>>><<<><<<<>>>><>>>><<>>>><<<><<<<><<>><>>>><<<>>>><<<<>>><>>>><<<><<>>><<>><<<>><>><<<>>><<>>>><<>>><<>>>><<><<>>>><<<<>>><<<<>>><<<>>><>><<><<<<>>>><<<>>><><<<<>>><<>>><<>><<<>><<<<><><<<<>>>><<<><<>>>><<<>>>><>><<<>>>><>>>><><<<<>><<<>>><>>><>>><<<>><<<<><>>><<>>><<<>>>><>>>><><<<<>>>><<>>><>>><<<<><<>>><<<><<<>>>><<<<><><<<<><<<<>><<<><>><<<<>>><>><<><<>>><<<>>><<<<>><<<>>><<<>>>><<<<>>><>><>><<<<>>>><>>><<<<>>><<>>><>><<<><>>>><<<<><<<>><><<<<>><<<<>>>><>><>><>>>><<<>>>><<>>>><<<>>><<<<>>>><<<<>>>><<><><<<>>><<>>><<<<>>>><<<<><<>>>><<<><<<<>>>><<>><<<>><<><>><<<<>>><<<<>>><<<<>>><<>>>><<<<>><<>>><<>>><>>>><<<>>>><<>>><<<<><<<>>><<<<><<><<>>><<>>>><<<<>>><<>>><<<<>>>><<<<>>><<<<><>><<<>>>><<<<>><<<>>><<<><<>>>><>>><<>>><>>>><<<><<<<><<<><<<>>>><<>>>><>>><>>>><<>>>><<<<>>>><>><<<<><<<>>>><><<>>>><<><<<>><<<<>>>><<<<><<<><<<<>>>><>><><<<<>><<<<>>><<>><<>>><>>><<<>>>><<<<>>><<<<>>><<>>>><<>><>><<<<>>>><<<>><<>><><><<>><<<<>>><<><<<<><><<<>>>><<<<>>><<<<>>>><<<>>><<<<><<<<>>><<><<><<>>><<<><<>><>>>><<<>><<>>><<<<>>>><>>>><<<>>>><<<<>><<><<<>>>><<<>>><>>>><<<<>>>><<<<><<>>><<<<>>>><<>>>><<<<><<<<>>><<>>><><<<<><<>>><>>>><<><<<>>>><>>><<<>><<<>><<<<>><>>>><<<>><<<<><>>>><<<>><<<<>>>><<<<>><<<<>><<>>><>>>><<>>><<<<><<>><>><<<<>>>><<<<>>>><<>><<<<><<<<><<>><><<>>>><<<<>>><<<<>><<<<>><<<<>><<<<>>><<<<>>><<<<><<<>>><><>>>><<>><>>><<>>><<<>>><<><>>>><<<<>>><<<>><<<<>>><<<>>><<>><<>><><<>><<>><<>>>><>>><<<>>>><<<<>>>><<<>><>>>><<<>>><<<><<>>>><<<>><<<<>><<<>>><<<<>><<><<<>>>><<>><>>>><<<<><>>>><<><<<<>><<>>><<>><<<<>>>><<>>><<>><<>>><>>>><<<>><<<><>><<<<>>><<<>>><<><<>>>><<<><>>>><>><<<>><><<<<>><<<<>>>><<<<>><><<<>>>><><<<<>>>><<>>><>>>><<>><>><<<><>><<<>>>><>><<<><<><><<<>>><<>><<<<>><<<>>><<>><<><>><<<<>>>><<<<>><<>>>><<<>><<><>>>><<<<>>>><>>><<>>><><<<<>><<>><>>>><<<<>>>><><<>>>><>>>><<<<>>>><<<<><>><><<<>>>><>>><<><><<<><<<<>>>><<<<>><<<>>>><<<<>>>><<<<>><<>>>><>><>>>><<<<><<>><<<><<<>>>><<<>>><<>>>><>><<<<>><<<>>>><<>>>><<<<>>>><<<<><<<>><>><><><<<><<<><<<><<<<>>><<>>><<>>><<<>><<<><<<<><<>>>><>><>><<<>>><<<<>>>><<<<>>><<<<>>><<<>>>><<>><<>><>><<>>><<>>>><<<<>>>><>>>><><><<<<><<<<>><<<><<<>>><<>>><<<>>>><<>>>><<>>><<<<>>>><<<><<<<>>><<<<><<>>>><<<<>>><>><><<>>><>>>><>>><<<>><<<<>><<<>>>><<>><<<>>>><<<><<<<>>><<<<><<<><<<>><<<>>>><<>>><>><<<<>>><<<>><>>>><<><><<<><><<<>>>><<<>><<><<<>>>><<<>>>><>>>><<<>>><<<>>><<<<>><<>><>><<>><>>><<><>><>>>><<<><<<><<<><<<<>><<>>>><>>>><>>><<<<><<<<>>>><<><<<>>><<<>>>><<<>>><<>>>><<>><><<<>><<<<>>>><<<<><<>><>><<<>><><<>><<<<>>><>>>><<<>><<<>><<<>>><<><<<><>><><><<>><<<>>><<<><<<><>>>><>>>><>>><<<<>>>><><<<<>>><<>><<<<>><<>>><>>>><<>>><>>>><<<>>>><<<>>>><<<<>>><<<<>><<<>>><<<>><<<>>><<<<>>><<>>>><<>>>><<<<>>>><<<<>><<<>>><<<<>>><>>>><<<<><<<<>><<<<>>>><>>><<<>><<>>>><<><<>><<<>>><<<<><<<<>>>><<<<><<<>>>><<<<>>><><<<<><<<<>><><<<>>>><>><<<<>><><>><<<>>>><><<>>>><><<<>>>><>>><<<<>>>><<<<>>><<<<>><<>>>><>>>><>>><<<><<>>>><<<<><<<<>><><>><<<>><<><<<<>>><<<<>>>><<<>><>><<>>><<<>>><<<<>>><<<><>>><<<>>><<<>><<<>>>><<<>><<>>>><<<<>>><<<<>><<<><>><<<>><><<<>><<<<><><>><<<><>><><<<>>><>>>><>>>><<<>><<<>>>><<<>>><<<<>>><<<>>>><<>>>><>><<><<<>><<>>>><>>>><<>><<<<><<<>><<>><<<><<>>>><<<<><<<>>>><<><<>>>><<><>>><><<>>><<<><<><<<<>>><<>>><>>>><><>>><<<<>><<>><<>>>><<<<>>><<<>>><<><<<><<<<>>><<>>><>><>>>><<><<<>>><<<<>>>><<>><<<<>><>>><<<>>><<>>>><<<>><<><<<>>><><<>>><<><<>>><>>>><<<<>>>><<<<>>>><<<>>>><<<<>>>><><<<<>>><>>><<<<>>><<>><<<>>>><>>>><><<<>><<>><<<>>><<<<>>><<<>>>><><<<<>>>><<><<>>><<<<>>>><<>>><<<<><>>><<<>>><<><<><<<>><>>>><<<>>>><<<>><<<>><<>>><<>>>><<<><>>>><<<<>>><<<<>><<>><<<>>><<<<>><<<<>>>><<>><<>><<<<>>>><>>>><<<>><>>><><>>><>>>><<<>>>><<>><<<>><<>><>>>><<<>>>><<<<><>>><<><<><>>>><<>>>><>>><<<<>>>><>>>><<<><<<><>>><<<>>><>>><<<>><<<<>>><>>>><<><<<<><<<>>>><<<<><<<<>><<>>><<<<><<<<>>><<>>><<<>>>><<>>>><<<>>>><<>><<>>><<<>>><<>>><>>><<>><<>><<<<><<>>>><<<>>>><<<>><<<<>>>><<>>>><>>>><>><>>>><><<<>>>><<<>>><><<<<>>><<<>><<<>><<<<>><<<>>>><<<<>>>><>>>><>><<<><><>>><>>>><<<<>>>><<<>>>><<>><>>>><<<<><<<>>>><>><<<>><<<<>>>><>>><<<<>>><<<<>><<<>>><<<<>>>><<>>><<>><<<>>><>><<>>>><<<<>>><<<<>>><<<<><<<<>>>><<<<><<<<>><>>><><<<>>><<<<>><<<<>>>><<<>>><>>><>>><<<>>><>>>><>><<>><<<<><<><>>>><<<<>>>><<<<>>>><<<>>>><><<<<>>>><<>>><<<>>>><<>>>><<<>>>><<<>><<<>><<>><<>>>><>>><>>>><<<<>>><<<>><<>>>><>>>><<<><<><<<>><<<<>>>><<<><<<<>><<><<<>><<<<><>><<<<><<>>>><>>>><<><<<<>><<>><<<>>>><<>>>><<<<>>><<><<<>>>><<<<>>>><<<>>><>>>><<>>><<<<>>><<<<>>><<<<>>>><<>>>><<>>>><<<<>>>><<>>>><<>>><>><<<<>>><>>>><>><<>>><<<<>><<<>><<><<>>>><<<>><<>><>><<<>>>><<><<><<><<><<>>>><<<>><<>>>><>>><<<<><<><>>><<<><<<<>>>><<<>>><<<<>><>>>><>><<<>>><<<>>><<>><<<><<<<>>>><<<><<><<<>>><<<<>><>>><>>><<<<>>><><<<<>><<>>>><>><<>>><<><<>>><<<<>>>><>><<<><<<>>>><<<>>>><<<>><<<>>>><<><<<<><<<<>><<>>>><>>><>>><<<<>><>><>><<<>>><<>><><>>><<<<><<<<>>>><>>><<<><<><>>><<<<>>>><><>><<<><<<>>><>>><<>><<>>>><><>><>><<<>><<>>>><<<>><<<<>>>><<<>><<>>><<<<>>>><<<<><<<>><<<><>>>><<<<>>>><<<>><<<><<<>><>>>><>>><<><>><<>>>><<<<><><<>>><<<<>>><<<<>>><<>>><<>><<<<>>>><<<<>>>><><><<>><>>>><<>>><<<<>>>><<>><<<>><<<<>>><<>>><><<>><<>><<<>>>><>><<><<<<>><<<>><<<>>>><>><<<<>><><>>>><<<<>><<<>>><<<<>><<<>><><>>>><<>><>>>><<<<>>>><<<<>>><<<<><<<<><<>>><<<<>>><<<<>>><<<<>>>><>><<><<>><<>>>><<<<>>><<<<>><<<>><>>><<<<><<>>><<<>>>><<><<<>>>><<><>><<><><<<><<<<>>><>>><<>>>><<<<><<>><>>>><<>>>><<<<>>>><<<<>>><><<<<>>>><<>><<>><<<>>>><<>>><>>><>>>><<<<>>><<>>><><>>><<><>>>><<<<>>><<<>><<>><>>>><<><<<>>><<<>>>><<<>>>><<>>>><<<<>>>><<<<><<>>><<>>>><<>>>><<><<<>><<<<><<<<>><<>>>><<><<<<>>><<<><<<<>>><<<>>>><<>>><>><<>>><>>><<>><>>><>>>><<>><<<><<>>>><>>>><<<><>>>><<<<>>>><<<<><><>>><<<>>>><><>>>><<<>><>>><<<>>>><<><<><<<<>>>><>>><<<>><<<><>><<<><><<<>>>><<>><<><<<>>>><<<>>><<<>>>><<<>>>><<<><<>><<<>><><<<>>>><<<<>>><<>><>>>><>><<>>>><<<><<>><<<><<<>>>><<<<><<<<><><<<>>>><><<<<><<<>>>><<<<>><<>><><<<>>><>><<<<>><<<>>>><>>>><<<>>><<<<><<<>>><<>>>><<<<>><<<<>>><><<><<<<><<>><>>><>><>><<<<>>>><><<>>><<<<>>>><<<<>>>><<<>><><<<<>><>>>><<>>><<<<>>><<<>>>><<>>>><<<<><<<<>>><<<>>><<>>>><<<>>>><<<>>>><<<<><<<<>>><<>>>><>>><>><<>><<<>><<<>><>><<>><<><<<<>>>><<<>><<>><<<<>>><<<><><>><<<><<>><<<<>><<>>>><<><<<<>><<<<>>><<<<>>>><<<<>><<<<>><<<>>><>>>><<<>>><<<<>>>><><<<>>><<>>>><<<><<<>><<>>><>>><<>>><>>><<>>><<<><<<>>>><<>><<>><<>>><<>><<<>>>><<<>>>><<<>>><<<><><>>>><<<<><<<>>><<<><>><<<<>>>><<<>><<<<>>><<<>><><<<>><<<<>>><<<><<<>>>><><<<<>>><<<>><<<>><<<<>>><<<<>>><<<<>>>><<>>>><<<<>>><>><<<><<<><>><<<>>><<<>><<<<>><<<<><<><>>>><<><<<>>><<<<>>>><<>><>>>><<<<>>>><<><<<<>><<<<>>><<<>>>><>>>><>><<>><<<><><<<<>>>><><>>><<<>>><<<<>>><<<<>>>><<><<<>>><<>>>><<<><>><><<<<>>>><<<<>>>><<<>>><<<>>>><<>>>><<<>>>><<<>>><>><<<<>><><>><<<<>>>><>>>><<<>><<>>>><<><<<<>>><>>><<<>>>><<<<><<><>>><<<<><<<<>>><<>><><<<<>>>><<<>><>>>><>>>><<>><<<>>>><<<<>>><<<>>>><<<>>>><<>>><<<>>><<<<>><<<<>><>>><<>>><<>>>><<<<>>>><>>><>><<><>>><<<><<>>>><><<<>>><<<><<>>>><<<>>><><<<<>>><<><>>><<>>><>>>><><>><<>>><<>>>><<>>>><<<<>>><>><><<<<>>>><<<<>>><><<>><<<<>><<>><<<>>><<<<>>><<<>>><<<<>><><<<><<<>>>><>>>><<<<>><<<>>><<<><<><<<><>><><<<>>>><<<<>>><<<<>>><<<<><<<>>>><<<<>>>><<<>>><>><><<>><<<>><<><<<<>><<><<<>>><<>>><<>>>><<>>>><><>>>><<<>>>><<<<><<>><<<<>>>><<<>><<<<>><><>>><<<<>>>><<>>><<<>><<<>>><<>>>><<<><<<<>><<>><<<>><<>>>><<>><<<<>>><>>><>><>>>><<<>>><<>>><<<>><<>>><<<<><<<><<>>><<<<>>><>><><<<<><>><>>><>><<><<<<><<>>><>><>><<>>><<>>><>>><<<<>>><<<<>>><<><>><<<<>>><<<<><<>><<<><><>>><>>><<<<>>><<<><<<>><<><>>>><<>><<>><<<<><<<<>><<>>>><>><<><<<>>>><<<>>>><<><<<<>>>><<<>><<<>>>><<<><>>>><<<>><<>><<<>><<<<>>>><<<>>><<<>><<<>>>><<<>>><<>>><<<<>>>><>>>><<>>>><><<<<>><<><<><<>><>>><<>>>><<>>><<<<><<><>><<<<><>><<<<><<<<>><<>>><<<<><>>>><>><<<>>>><<<<>>>><<>><<<<>>><<>>>><<>>><<<<>><<<><<<<>><<<>>><<<>>>><<<>><>><<>>>><<><<><>><>>><>><<<>>><<>>><<<><<<<>>>><<<<>>><<<<>>><<>>><<<<>>>><<><><><>><<>>><<<>>><<<<><>>><>><>><<<<><<<<>><<<<>>><>>>><<<>>><<<>>>><<<>>>><<<<>>>><>>><<<<>>>><><<<><<>>><<<<>>><<>><<>><<<<>>>><<>><><<<<>><<<>>><<<>>><<<<>><<<>>>><<<<>>>><>>><>>>><<>>><<><<><<<>>><<<><<<><>>>><<<<><<>>><<<>><<<<><<<>>><><<<<>>>><<>>>><>>><>><>>>><<<<><<<><<<><<><<>>>><<>>><>>>><<<><<>>><<<>><<><<<><<<><<<>><>>><<<>><>>><<>><>><<<<>>>><<<<>>>><><<<<><<><<<<>>>><>>><<<>>>><>>>><<<>>><<<>>>><<<>>><<<>>>><>>><<<>><>>><<>>>><><<<>><<<><>>>><<<>>>><<<<><<<>>>><>>><<<>><<>><<<<>><<<><<>>><<<>>>><>>>><<<<>>><<<>><>>>><><<<<><<>>>><>>>><<<><<>>>><<<<>>><>>><>>>><><<>>>><<><><><<>><<>>><<<><<>>>><<><<>><<>>>><<<>>>><<><<><<>>><<<>><<<>>>><><<<>>>><<<<><>>>><<<>>><<>><>>>><<<<><<>>><<><>><'

In [63]:
# changed the return for part 2
def get_wind():
    index = 0
    while True:
        yield index, wind_input[index]
        index += 1
        if index == len(wind_input):
            index = 0

In [54]:
# helper function to print the chasm and pieces in a readable way
def print_chasm(c, ht=0, bot=0, piece=None):
    coords = set()
    if piece:
        coords = set(piece.get_coords())
        
    if ht == 0:
        ht = len(c)
    else:
        ht += 1
        
    for i, r in enumerate(reversed(c[bot:ht])):
        for j, c in enumerate(r):
            if(ht-i-1, j) in coords:
                print('@', end='')
            else:
                print(c, end='')
        if (ht-i-1) % 5 == 0:
            print(' ',abs(ht-i-1))
        else:
            print('')

In [31]:
def place_piece(p, h, offset):
    coords = p.get_coords()
    for y, x in coords:
        chasm[y-offset][x] = '#'
        if y > h:
            h = y
    return h

In [90]:
# code for part 2, sliding window ... tqdm estimates this will complete in 3500 hours ... 145 days :(
check_num_blocks = 1000000000000
chasm = [[] for _ in range(0, 100000)] 
for i, _ in enumerate(chasm):
    chasm[i] = ['|'] + ['.' for _ in range(0,7)] + ['|']
chasm[0] = ['+'] + ['-' for _ in range(0,7)] + ['+']


highest_used = 0
wind = get_wind()
ht_offset = 0

type_and_index_to_sequence_num = {} # dict for tracking the repeating cycle
found_repeat = False
repeat_tuple = ()

for count in tqdm(range( 1, check_num_blocks+1 )):
    b = block(count, highest_used)
    
    # print data needed to solve the problem, array populated based on finding cycle start and length
    if count in [ 9, 10, 11, 795+10, 794+10, 796+10]:
        print(count, highest_used)
    moving = True
    while moving:
        
        # find the cycle for part 2
        windex, w = next(wind)
        if (b.shape, windex) not in type_and_index_to_sequence_num.keys():
            type_and_index_to_sequence_num[(b.shape, windex)] = count
        elif found_repeat and (b.shape, windex) == repeat_tuple:
            print('C:', count, 'S:', b.shape, 'WI:', windex, 'first appeared at count', type_and_index_to_sequence_num[(b.shape, windex)], 'height:', highest_used)
        else:
            if not found_repeat:
                repeat_tuple = (b.shape, windex)
                found_repeat = True
                
        # part 1 code continues here
        
        b.move(chasm, w, ht_offset)
        moving = b.move(chasm, 'down', ht_offset)
    highest_used = place_piece(b, highest_used, ht_offset)
    
    # cut off the bottom and allocate more space above
    if highest_used > 97000 + ht_offset:
        chasm = chasm[50000:] + [['|'] + ['.' for _ in range(0,7)] + ['|'] for _ in range(0, 50000)] 
        ht_offset += 50000
        

print(highest_used)



  0%|          | 0/1000000000000 [00:00<?, ?it/s]

9 18
10 22
11 22
804 1310
805 1314
806 1316
C: 3410 S: 0 WI: 52 first appeared at count 10 height: 5386
C: 5105 S: 0 WI: 52 first appeared at count 10 height: 8057
C: 6800 S: 0 WI: 52 first appeared at count 10 height: 10728
C: 8495 S: 0 WI: 52 first appeared at count 10 height: 13399
C: 10190 S: 0 WI: 52 first appeared at count 10 height: 16070
C: 11885 S: 0 WI: 52 first appeared at count 10 height: 18741
C: 13580 S: 0 WI: 52 first appeared at count 10 height: 21412
C: 15275 S: 0 WI: 52 first appeared at count 10 height: 24083
C: 16970 S: 0 WI: 52 first appeared at count 10 height: 26754
C: 18665 S: 0 WI: 52 first appeared at count 10 height: 29425
C: 20360 S: 0 WI: 52 first appeared at count 10 height: 32096
C: 22055 S: 0 WI: 52 first appeared at count 10 height: 34767
C: 23750 S: 0 WI: 52 first appeared at count 10 height: 37438
C: 25445 S: 0 WI: 52 first appeared at count 10 height: 40109
C: 27140 S: 0 WI: 52 first appeared at count 10 height: 42780
C: 28835 S: 0 WI: 52 first appea

KeyboardInterrupt: 

## sample data:
first instance of cycle appears at count 9, with height 16

then repeats every 35 counts. Cycle adds 53 height per iteration.

target is 1000000000000

1000000000000 - 9 is how many iterations after cycle starts

/ 35 = 28571428571 and change

% 35 = 6

height at 9+6 = 37

so height should be (height at start of cycle) + (cycle height * 28571428571) + (cycle height at start+6 - height at start)
15 + (53*28571428571) + (23-15) = 1514285714286  !! very close to answer of 1514285714288 has an off by one, so only two anwers. check it and see!


8 13  <-maybe?
9 15  <- here
10 17
14 23 
15 23 <- here
16 25 <- maybe?
C: 44 S: 4 WI: 37 first appeared at count 9 height: 70
C: 79 S: 4 WI: 37 first appeared at count 9 height: 123
C: 114 S: 4 WI: 37 first appeared at count 9 height: 176
C: 149 S: 4 WI: 37 first appeared at count 9 height: 229
C: 184 S: 4 WI: 37 first appeared at count 9 height: 282

betting last bit of equation is actually 25-15 rather than 23. Feels like the right place intuitively, though I can't say exactly why.

After a few minutes of thought, the why is obvious. The print statement is from the beginning of the cycle and does not account for the piece dropped in the 16th tick.




In [93]:
15 + (53*28571428571) + (23-15)

1514285714286

actual numbers from puzzle set:

9 18
10 22
11 22
804 1310
805 1314
806 1316
C: 3410 S: 0 WI: 52 first appeared at count 10 height: 5386
C: 5105 S: 0 WI: 52 first appeared at count 10 height: 8057
C: 6800 S: 0 WI: 52 first appeared at count 10 height: 10728
C: 8495 S: 0 WI: 52 first appeared at count 10 height: 13399
C: 10190 S: 0 WI: 52 first appeared at count 10 height: 16070


cyclic period is 1695 cycles, and accumulates 2671 height per.
starts on cycle 10

(tot-start) / 1695 = 589970501.4690266
(tot-start) % 1695 = 795


so height should be (height at start of cycle) + (cycle height * 589970501) + (cycle height at (start+795) - height at start)

yup. Was right, off by one in the last part, is (start+796). Got it in first guess. (intuitiion was good. Reasoning above.)

In [94]:
22 + (2671*589970501) + (1316-22)

1575811209487